In [ ]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN, WD_LINE_SPACING
from dataclasses import dataclass
from typing import List, Tuple
import textwrap
import logging

logging.basicConfig(level=logging.INFO)

@dataclass
class SummaryItem:
    """Simple data structure for summary content items"""
    content: str
    is_continuation: bool = False

class SummaryContentGenerator:
    def __init__(self, template_path: str):
        self.prs = Presentation(template_path)
        self._validate_template()
        self.current_slide_index = 0
        self.LINE_HEIGHT = Pt(9)
        self.ROWS_PER_SECTION = self._calculate_max_rows()
        self.CHARS_PER_ROW = 55  # Slightly more for summary content
        self.WHITE_COLOR = RGBColor(255, 255, 255)
        
    def _calculate_max_rows(self):
        """Dynamically calculate max rows based on template shape"""
        slide = self.prs.slides[0]
        shape = next(s for s in slide.shapes if s.name == "coSummaryShape")
        return int(shape.height / self.LINE_HEIGHT) - 2  # Account for margins
        
    def _validate_template(self):
        """Validate that required shape exists in template"""
        required_shapes = {
            0: ["coSummaryShape"],
            1: ["coSummaryShape"] if len(self.prs.slides) > 1 else []
        }
        
        for slide_idx, slide in enumerate(self.prs.slides[:2]):  # Only check first 2 slides
            if slide_idx in required_shapes and required_shapes[slide_idx]:
                missing = [name for name in required_shapes[slide_idx]
                          if not any(s.name == name for s in slide.shapes)]
                if missing:
                    raise ValueError(f"Missing coSummaryShape on slide {slide_idx+1}")

    def parse_markdown(self, md_content: str) -> List[SummaryItem]:
        """Parse markdown content into simple summary items"""
        items = []
        
        for line in md_content.strip().split('\n'):
            stripped_line = line.strip()
            
            # Skip empty lines
            if not stripped_line:
                continue
                
            # Remove markdown formatting (headers, bullets, etc.)
            cleaned_line = self._clean_markdown_line(stripped_line)
            
            if cleaned_line:
                # Split long lines into wrapped chunks
                wrapper = textwrap.TextWrapper(
                    width=self.CHARS_PER_ROW,
                    break_long_words=True,
                    replace_whitespace=False
                )
                chunks = wrapper.wrap(cleaned_line)
                
                for chunk in chunks:
                    items.append(SummaryItem(chunk))
        
        return items
    
    def _clean_markdown_line(self, line: str) -> str:
        """Remove markdown formatting from a line"""
        # Remove headers
        line = line.lstrip('#').strip()
        
        # Remove bullet points
        line = line.lstrip('*-+').strip()
        
        # Remove bold/italic markers
        line = line.replace('**', '').replace('*', '')
        line = line.replace('__', '').replace('_', '')
        
        # Remove links (basic cleanup)
        import re
        line = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', line)
        
        return line.strip()

    def _plan_content_distribution(self, items: List[SummaryItem]) -> List[Tuple[int, List[SummaryItem]]]:
        """Plan content distribution across maximum 2 pages"""
        distribution = []
        content_queue = items.copy()
        slide_idx = 0
        max_slides = 2
        
        while content_queue and slide_idx < max_slides:
            page_items = []
            lines_used = 0
            
            while content_queue and lines_used < self.ROWS_PER_SECTION:
                item = content_queue[0]
                item_lines = self._calculate_item_lines(item.content)
                
                if lines_used + item_lines <= self.ROWS_PER_SECTION:
                    # Mark continuation if not first slide and first item
                    if slide_idx > 0 and not page_items:
                        item.is_continuation = True
                    
                    page_items.append(item)
                    content_queue.pop(0)
                    lines_used += item_lines
                else:
                    # Split content if possible
                    if lines_used < self.ROWS_PER_SECTION:
                        available_lines = self.ROWS_PER_SECTION - lines_used
                        split_content = self._split_content(item.content, available_lines)
                        
                        if split_content['first_part']:
                            if slide_idx > 0 and not page_items:
                                split_item = SummaryItem(split_content['first_part'], True)
                            else:
                                split_item = SummaryItem(split_content['first_part'])
                            page_items.append(split_item)
                            
                            # Update remaining content
                            content_queue[0] = SummaryItem(split_content['remaining'])
                            lines_used = self.ROWS_PER_SECTION
                    break
            
            if page_items:
                distribution.append((slide_idx, page_items))
            slide_idx += 1
        
        return distribution

    def _calculate_item_lines(self, content: str) -> int:
        """Calculate lines needed for content"""
        wrapper = textwrap.TextWrapper(
            width=self.CHARS_PER_ROW,
            break_long_words=True,
            replace_whitespace=False
        )
        return len(wrapper.wrap(content))

    def _split_content(self, content: str, max_lines: int) -> dict:
        """Split content based on available lines"""
        wrapper = textwrap.TextWrapper(
            width=self.CHARS_PER_ROW,
            break_long_words=True,
            replace_whitespace=False
        )
        
        wrapped_lines = wrapper.wrap(content)
        
        if len(wrapped_lines) <= max_lines:
            return {'first_part': content, 'remaining': ''}
        
        first_lines = wrapped_lines[:max_lines]
        remaining_lines = wrapped_lines[max_lines:]
        
        return {
            'first_part': ' '.join(first_lines),
            'remaining': ' '.join(remaining_lines)
        }

    def _get_summary_shape(self, slide):
        """Get the coSummaryShape from slide"""
        shape = next((s for s in slide.shapes if s.name == "coSummaryShape"), None)
        if not shape:
            raise ValueError("coSummaryShape not found in slide")
        return shape

    def _populate_summary_content(self, shape, items: List[SummaryItem]):
        """Populate the summary shape with formatted content"""
        tf = shape.text_frame
        tf.clear()
        
        # Set text frame properties
        tf.word_wrap = True
        
        for idx, item in enumerate(items):
            # Add paragraph
            if idx == 0:
                p = tf.paragraphs[0] if tf.paragraphs else tf.add_paragraph()
            else:
                p = tf.add_paragraph()
            
            # Add run with content
            run = p.add_run()
            
            # Add continuation marker if needed
            if item.is_continuation and idx == 0:
                run.text = f"(continued) {item.content}"
            else:
                run.text = item.content
            
            # Apply formatting
            run.font.name = 'Arial'
            run.font.size = Pt(9)
            run.font.bold = True
            run.font.color.rgb = self.WHITE_COLOR
            
            # Apply paragraph formatting
            pf = p.paragraph_format
            pf.line_spacing_rule = WD_LINE_SPACING.SINGLE
            pf.space_before = Pt(0)
            pf.space_after = Pt(0)
            pf.left_indent = Inches(0.07)

    def generate(self, md_content: str, output_path: str):
        """Generate PowerPoint with summary content"""
        try:
            items = self.parse_markdown(md_content)
            distribution = self._plan_content_distribution(items)
            
            if not distribution:
                logging.warning("No content to generate")
                return
            
            # Populate slides with content
            for slide_idx, page_items in distribution:
                if slide_idx >= len(self.prs.slides):
                    logging.warning(f"Insufficient slides in template. Content truncated at slide {slide_idx}")
                    break
                    
                slide = self.prs.slides[slide_idx]
                shape = self._get_summary_shape(slide)
                self._populate_summary_content(shape, page_items)
            
            # Remove unused slides if template has more than needed
            slides_needed = len(distribution)
            if len(self.prs.slides) > slides_needed:
                unused_slides = list(range(slides_needed, len(self.prs.slides)))
                self._remove_unused_slides(unused_slides)
            
            self.prs.save(output_path)
            logging.info(f"Successfully generated summary with {len(distribution)} slides")
            
        except Exception as e:
            logging.error(f"Summary generation failed: {str(e)}")
            raise

    def _remove_unused_slides(self, slide_indices):
        """Remove unused slides (indices in descending order)"""
        for slide_idx in sorted(slide_indices, reverse=True):
            if slide_idx < len(self.prs.slides):
                xml_slides = self.prs.slides._sldIdLst
                slides = list(xml_slides)
                
                # Remove relationship
                rId = slides[slide_idx].rId
                self.prs.part.drop_rel(rId)
                
                # Remove from slide list
                xml_slides.remove(slides[slide_idx])
                
                logging.info(f"Removed unused slide {slide_idx + 1}")

# Sample markdown content for testing
sample_md_content = """
# Executive Summary
This comprehensive analysis provides key insights into the current market conditions and strategic recommendations for the upcoming fiscal quarter.

## Key Performance Indicators
Revenue growth has exceeded expectations by 15% compared to the previous quarter, driven primarily by strong performance in the technology and healthcare sectors.

Market penetration in emerging markets has shown significant improvement, with particular strength in the Asia-Pacific region contributing to overall portfolio diversification.

## Strategic Recommendations
Implementation of digital transformation initiatives should be prioritized to maintain competitive advantage and improve operational efficiency across all business units.

Investment in research and development capabilities will be crucial for sustaining long-term growth and innovation in core product offerings.

## Risk Assessment
Current market volatility presents both challenges and opportunities that require careful monitoring and adaptive strategy implementation.

Supply chain dependencies need to be evaluated and diversified to mitigate potential disruptions in the global logistics network.

## Financial Outlook
Projected revenue targets remain achievable based on current market trends and pipeline analysis, with conservative estimates suggesting continued growth trajectory.

Cost optimization initiatives are expected to improve margins while maintaining quality standards and customer satisfaction levels.

## Conclusion
The strategic position remains strong with multiple growth opportunities identified for the next planning period, requiring focused execution and resource allocation.
"""

# Usage example
if __name__ == "__main__":
    generator = SummaryContentGenerator("template.pptx")
    
    try:
        generator.generate(sample_md_content, "summary_report.pptx")
        print("Summary generation completed successfully!")
    except Exception as e:
        logging.error(f"Generation failed: {str(e)}")
